# Prepare a protein-ligand system for simulation.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

This is part 1 of a four-part tutorial on molecular dynamics simulations of biomolecular systems prepared for *CompBioAsia* 2025.

* In this first part you will learn how to take an initial model for a protein-ligand complex and process it to get to a state where it is ready for MD simulation.

* In part 2 you will learn how to perform MD simulations on this using the AMBER MD package.

* In part 3 you will learn how to perform MD simulations on this using OpenMM instead.

* In part 4 you will learn how to perform MD simulations on this using GROMACS instead.

The molecular system we will be using for these tutorials is a complex between the anticancer drug [Imatinib](https://en.wikipedia.org/wiki/Imatinib) and it's target protein, the [Abl tyrosine kinase](https://en.wikipedia.org/wiki/ABL_(gene)).

**Authors**:
This tutorial is adapted from CCPBioSim's [BioSim analysis workshop](https://github.com/CCPBioSim/BioSim-analysis-workshop).

*Updates*: Charlie Laughton (charles.laughton@nottingham.ac.uk)


## Learning outcomes
* Understand the steps involved in preparing a simulation-ready system from crystal structure-derived data
* Have a working knowledge of some of the software tools that can help with various stages in the process

## Prerequisites
Assuming you have started this Notebook using the `run_notebook.sh` script in this folder, your Python environment should be complete.

**However**, it also requires that you have [`Chimera`](https://www.cgl.ucsf.edu/chimera/download.html) installed and that you can start it from a terminal window using the command *chimera* (or similar). Unfortunately there is currently a bug in the newer `ChimeraX` that means it can't be used here.

# 1. Construct a complete molecular model for the protein-ligand complex

The Protein Data Bank ([PDB](www.rcsb.org)) is a very valuable source of structures for MD simulation, but it must be understood that the crystal structure data itself is really just raw material - there are typically many steps that must be taken in order to generate simulation-ready systems from it. Some of these are:

1. The crystal structure may contain more data than is needed for the simulation (e.g. multiple copies of the protein) - it may need to be edited down.

2. Almost certainly the crystal structure will have missing data. It possible that certain heavy atoms - maybe whole sections of the protein - were not resolved in the experiment and are missing. Molecular simulations require chemically-complete models for the components so this must be rectified.

3. Even if the structure is complete at the heavy-atom level, if it was solved by Xray crystallography it is unlikely that any hydrogen atoms will have been resolved. so these missing atoms must be added as well.

There are very many approaches to system preparation for MD, what you see here is just one of them. It leverages a number of different system preparation tools from different sources, so to make the process simpler these have been "wrapped" into a small number of Python functions in the package *cba_tools*.

If you want to see the details, take a look at `cba_tools.py`!


## 1.1 Import required packages

In [ ]:
import mdtraj as mdt
import nglview
from cba_tools import fix, add_h, param

## 1.2 From Xray crystallography data to a partial structural model

Although it's now often possible to obtain a (nearly) "ready to run" model for any protein via Alphafold (or similar), it still remains the case that if a good quality and relevant crystal structure is available from the Protein Data Bank this can produce a better starting model for a simulation.

We won't go into the process of identifying such a candidate, but take it as read that the structure with PDB code [2HYY](https://www.rcsb.org/structure/2HYY) turns out to be suitable.

A copy of this PDB file is included ('2hyy.pdb'),  step one is to take a look.

In [ ]:
pdb2hyy = mdt.load('2hyy.pdb')
view = nglview.show_mdtraj(pdb2hyy)
view.add_representation('ball+stick', 'water')
view

You should be able to work out that the crystal structure features four copies of the Abl protein, each with one molecule of Imatinib bound to it. Each has it's own collection of water molecules too. For now, we are going to assume that it's only the first of these copies (chain 'A') that we want to start building our simulation system.

What you probably can't see straight away is that there are some significant issues with this crystal structure. There are quite a few atoms that ought to be there, but aren't because they could not be seen in the experimental electron density. This results in some amino acid side chains being incomplete (e.g. a histidine sidechain - left panel below), and even some entire residues being absent, creating 'gaps' in the protein chain (right). 

|Missing side chain atoms|Missing residue|
|---------------------|------------------|
|![histidine](his.png)|![a gap](gap.png) |


For simulation purposes, these missing atoms must be reintroduced somehow. The CBA tool *fix*, which uses [pdbfixer](https://github.com/openmm/pdbfixer), can be used to do this. It requires the name of the PDB file to fix, a name for the 'fixed' file, a list of the chains to keep, and a decision as to whether any missing residues at the N- and C-terminii should be ignored (trim=True), or reconstructed as well (trim=False).

In the cell below you can see we have decided to use chain 'A", won't bother to reconstruct missing N- and C-terminal residues, and will call the remediated PDB file 'abl_imatinib_heavy.pdb' (because it should be complete at the heavy atom level, though still missing hydrogens):

In [ ]:
fix('2hyy.pdb', 'abl_imatinib_heavy.pdb', keep_chains=['A'], trim=True)

Let's take a look at the result:

In [ ]:
abl_sti = mdt.load('abl_imatinib_heavy.pdb')
view = nglview.show_mdtraj(abl_sti)
view

Take a good look, and convince yourself that the process has worked. Be aware that though `fix` works pretty well, it's not guaranteed to be perfect every time. You should always check the produced structure very carefully.

## 1.3 Completion of the structural model - addition of hydrogen atoms.

The next step is to add hydrogen atoms. There are a variety of methods to do this more-or-less automatically, but none is perfect. Here we will use the CBA tool *add_h*, which in turn uses the [addh](https://www.cgl.ucsf.edu/chimerax/docs/user/commands/addh.html) utility built into *Chimera*. 

It requires the names for the input and output PDB files, the name for the command which launches *Chimera*, and a decision as to whether you want the protein residue names to be modified to fit AMBER conventions or not (e.g. change the names of histidine residues from HIS to HID, HIE, or HIP according to what hydrogen atoms have been added to them).

In [ ]:
add_h('abl_imatinib_heavy.pdb', 'abl_imatinib_amber.pdb', 
      chimera='/Applications/Chimera.app/Contents/MacOS/chimera',
      mode='amber')

Now check the result visually:

In [ ]:
abl_sti_amber = mdt.load('abl_imatinib_amber.pdb')
view2 = nglview.show_mdtraj(abl_sti_amber)
view2

If you zoom in on the ligand, you should be able to see the hydrogen atoms now attached to it. Look carefully and you will see that both nitrogen atoms in the piperidine ring are predicted to be protonated, so overall the imatinib molecule has a charge of +2. If you click on one of the imatinib atoms, you will also see that the residue name for it is 'STI'. You will need both of these bits of information for the next step.  

Although not shown, all protein hydrogen atoms have also been added.

Again the same caveats as last time: `addh` generally does a good job but it's not guaranteed. Check the resulting structure very carefully.

# 2. Parameterization of the model

Now that we have a chemically-complete model for the protein and ligand, we can move on to the parameterization stage.

For this we will be using tools from the [AMBER MD](https://ambermd.org/) simulation package. Parameterizing the protein component of the system is easy, beacuse AMBER comes with a library of parameters for all "standard" biomolecular components (amino acids, nucleic acids, certain ions, solvants and lipids, etc.). But it has no knowledge of the parameters required for the Imatinib molecule in our system so we have to generate these ourselves.

The CBA tool *param* will do this for you. It requires:

 - the name of the PDB format file to process
 - the name of the AMBER parameter ("prmtop") file to generate
 - the name of the AMBER cordinates ("inpcrd") file to generate
 - the names of all non-standard residues ("heterogens") that will need to be parameterized
 - the formal charge on each of the heterogens
 - the type of solvent (water) box to add (see below)
 - the width of the solvent margin between the solute and the box boundaries

In addition, for more advanced use you can specify which forcefields you want to be used (otherwise defaults are selected automatically).


The options for the periodic box of solvent are "box", "cube", and "oct" (truncated octahedron). The figure below summarizes the differences:

![boxes](boxes.png)

"Box" adds the least solvent to satisfy the "buffer" criterion (white arrows), but if the solute (orange) rotates in the box, it may extend beyond it. "Cube" solves this, but means adding more water (so more atoms and a slower simulation). "Oct" reduces the number of waters required but still is safe for rotation of the solute.

In [ ]:
param('abl_imatinib_amber.pdb', 'abl_imatinib.prmtop', 'abl_imatinib.inpcrd', 
      het_names=['STI'], het_charges=[2],
      solvate='oct', buffer=10.0)

(This cell may take quite a time to run.)

You will notice that in addition to the files 'abl_imatinib.prmtop' and 'abl_imatinib.inpcrd', the process has generated two extra files: 'STI.mol2' and 'STI.frcmod'. These contain the parameterization data for the imatinib molecule that was generated as part of the process. 

You don't need these to run future simulations, but, as before, `param` is not perfect and it is a good idea to check them (the .mol2 file contains information about the atom and bond types assigned, the .frcmod file lists non-standard force field parameters that have been inferred). 

Visualize the result, which is your 'simulation-ready' system:

In [ ]:
system = mdt.load('abl_imatinib.inpcrd', top='abl_imatinib.prmtop')
view3 = nglview.show_mdtraj(system)
view3.add_representation('line', 'HOH')
view3

As you rotate the view you will be able to appreciate the shape and size of the periodic box of water that has been added. You can re-run `param` with 'solvate="box"' or 'solvate="cube"' to see the effect of these options, and experiment with changing the value of "buffer" too (it's in Angstroms).

# Next Steps
In the next workbook, you will use these files to start some MD simulations using AMBER.